In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.ensemble import GradientBoostingRegressor
from pandas_profiling import ProfileReport

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
xtrain = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-5/train.csv')
xtest = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-5/test.csv')
xsubmission = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-5/submission.csv')

In [ ]:
xtrain['Date'].max()

In [ ]:
#test_date_min = xtest['Date'].min()
#test_date_max = xtest['Date'].max()
#xtrain = xtrain[xtrain['Date']<test_date_min]

In [ ]:
xtrain['Date'].max()

In [ ]:
sum(n<0 for n in xtrain.TargetValue)

In [ ]:
#xtrain.TargetValue = [0 if i < 0 else i for i in xtrain.TargetValue]

In [ ]:
sum(n<0 for n in xtrain.TargetValue)

In [ ]:
xtrain.rename(columns={'Country_Region':'Country'}, inplace=True)
xtest.rename(columns={'Country_Region':'Country'}, inplace=True)

xtrain.rename(columns={'Province_State':'State'}, inplace=True)
xtest.rename(columns={'Province_State':'State'}, inplace=True)

xtrain['Date'] = pd.to_datetime(xtrain['Date'], infer_datetime_format=True)
xtest['Date'] = pd.to_datetime(xtest['Date'], infer_datetime_format=True)

xtrain.info()
xtest.info()

EMPTY_VAL = "EMPTY_VAL"

def fillState(state, country):
    if state == EMPTY_VAL: return country
    return state

def fillCounty(county, country):
    if county == EMPTY_VAL: return country
    return county

In [ ]:
X_xTrain = xtrain.copy()

X_xTrain['State'].fillna(EMPTY_VAL, inplace=True)
X_xTrain['State'] = X_xTrain.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

X_xTrain['County'].fillna(EMPTY_VAL, inplace=True)
X_xTrain['County'] = X_xTrain.loc[:, ['County', 'Country']].apply(lambda x : fillCounty(x['County'], x['Country']), axis=1)

X_xTrain.loc[:, 'Date'] = X_xTrain.Date.dt.strftime("%m%d")
X_xTrain["Date"]  = X_xTrain["Date"].astype(int)

X_xTrain.head()

#X_Test = df_test.loc[:, ['State', 'Country', 'Date']]
X_xTest = xtest.copy()

X_xTest['State'].fillna(EMPTY_VAL, inplace=True)
X_xTest['State'] = X_xTest.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

X_xTest['County'].fillna(EMPTY_VAL, inplace=True)
X_xTest['County'] = X_xTest.loc[:, ['County', 'Country']].apply(lambda x : fillCounty(x['County'], x['Country']), axis=1)

X_xTest.loc[:, 'Date'] = X_xTest.Date.dt.strftime("%m%d")
X_xTest["Date"]  = X_xTest["Date"].astype(int)

X_xTest.head()

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

X_xTrain.Country = le.fit_transform(X_xTrain.Country)
X_xTrain['State'] = le.fit_transform(X_xTrain['State'])
X_xTrain['County'] = le.fit_transform(X_xTrain['County'])

X_xTrain.head()

X_xTest.Country = le.fit_transform(X_xTest.Country)
X_xTest['State'] = le.fit_transform(X_xTest['State'])
X_xTest['County'] = le.fit_transform(X_xTest['County'])

X_xTest.head()

xtrain.head()
xtrain.loc[xtrain.Country == 'Afghanistan', :]
xtest.tail()

In [ ]:
xtrain1 = X_xTrain[X_xTrain.Target=='ConfirmedCases']
xtrain1.pop('Target')
xtrain1 = xtrain1.rename(columns={'TargetValue':'ConfirmedCases'})

xtrain2 = X_xTrain[X_xTrain.Target=='Fatalities']
xtrain2.pop('Target')
xtrain2 = xtrain2.rename(columns={'TargetValue':'Fatalities'})

xtest1 = X_xTest[X_xTest.Target=='ConfirmedCases']
xtest1.pop('Target')
xtest1 = xtest1.rename(columns={'TargetValue':'ConfirmedCases'})

xtest2 = X_xTest[X_xTest.Target=='Fatalities']
xtest2.pop('Target')
xtest2 = xtest2.rename(columns={'TargetValue':'Fatalities'})

xtest1

In [ ]:
from sklearn.ensemble import RandomForestRegressor



In [ ]:
countries = X_xTrain.Country.unique()

xout1 = pd.DataFrame({'ForecastId': [], 'ConfirmedCases': []})

for country in countries:
    states = X_xTrain.loc[X_xTrain.Country == country, :].State.unique()
    #print(country, states)
    # check whether string is nan or not
    for state in states:
        X_xTrain_CS = xtrain1.loc[(xtrain1.Country == country) & (xtrain1.State == state), ['State', 'County', 'Country', 'Date', 'ConfirmedCases']]
        
        y1_xTrain_CS = X_xTrain_CS.loc[:, 'ConfirmedCases']
        
        X_xTrain_CS = X_xTrain_CS.loc[:, ['State', 'County', 'Country', 'Date']]
        
        
        X_xTest_CS = xtest1.loc[(xtest1.Country == country) & (xtest1.State == state), ['State', 'County', 'Country', 'Date', 'ForecastId']]
        
        X_xTest_CS_Id = X_xTest_CS.loc[:, 'ForecastId']
        X_xTest_CS = X_xTest_CS.loc[:, ['State', 'County', 'Country', 'Date']]
    
        
        models = []
        for i in range(20):
         #   print('training model #',{i},'....')
            mid_model = RandomForestRegressor(n_jobs=-1)
            mid_model.fit(X_xTrain_CS, y1_xTrain_CS)
            models.append(mid_model)
          #  print('....done!')
        
        p_mid = np.zeros((X_xTest_CS.shape[0],))
        for mid_model in models:
            p_mid += mid_model.predict(X_xTest_CS)/len(models)
        
        xdata1 = pd.DataFrame({'ForecastId': [str(i)+'_0.05' for i in X_xTest_CS_Id.values.tolist()], 'ConfirmedCases': 0.05 * p_mid})
        xout1 = xout1.append(xdata1)
        xdata2 = pd.DataFrame({'ForecastId': [str(i)+'_0.5' for i in X_xTest_CS_Id.values.tolist()], 'ConfirmedCases': p_mid})
        xout1 = xout1.append(xdata2)
        xdata3 = pd.DataFrame({'ForecastId': [str(i)+'_0.95' for i in X_xTest_CS_Id.values.tolist()], 'ConfirmedCases': 2 * p_mid})
        xout1 = xout1.append(xdata3)

xout1.reset_index(inplace=True)
xout1 = xout1.rename(columns={'ConfirmedCases':'TargetValue'})

In [ ]:
countries = X_xTrain.Country.unique()

xout2 = pd.DataFrame({'ForecastId': [], 'Fatalities': []})

for country in countries:
    states = X_xTrain.loc[X_xTrain.Country == country, :].State.unique()
    #print(country, states)
    # check whether string is nan or not
    for state in states:
        X_xTrain_CS = xtrain2.loc[(xtrain2.Country == country) & (xtrain2.State == state), ['State', 'County','Country', 'Date', 'Fatalities']]
        
        y1_xTrain_CS = X_xTrain_CS.loc[:, 'Fatalities']
        
        X_xTrain_CS = X_xTrain_CS.loc[:, ['State', 'County','Country', 'Date']]
        
        
        X_xTest_CS = xtest2.loc[(xtest2.Country == country) & (xtest2.State == state), ['State', 'County','Country', 'Date', 'ForecastId']]
        
        X_xTest_CS_Id = X_xTest_CS.loc[:, 'ForecastId']
        X_xTest_CS = X_xTest_CS.loc[:, ['State', 'County','Country', 'Date']]
        
        models = []
        for i in range(20):
         #   print('training model #',{i},'....')
            mid_model = RandomForestRegressor(n_jobs=-1)
            mid_model.fit(X_xTrain_CS, y1_xTrain_CS)
            models.append(mid_model)
          #  print('....done!')
        
        p_mid = np.zeros((X_xTest_CS.shape[0],))
        for mid_model in models:
            p_mid += mid_model.predict(X_xTest_CS)/len(models)

        xdata1 = pd.DataFrame({'ForecastId': [str(i)+'_0.05' for i in X_xTest_CS_Id.values.tolist()], 'Fatalities': 0.05 * p_mid})
        xout2 = xout2.append(xdata1)
        xdata2 = pd.DataFrame({'ForecastId': [str(i)+'_0.5' for i in X_xTest_CS_Id.values.tolist()], 'Fatalities': p_mid})
        xout2 = xout2.append(xdata2)
        xdata3 = pd.DataFrame({'ForecastId': [str(i)+'_0.95' for i in X_xTest_CS_Id.values.tolist()], 'Fatalities': 2 * p_mid})
        xout2 = xout2.append(xdata3)

xout2.reset_index(inplace=True)
xout2 = xout2.rename(columns={'Fatalities':'TargetValue'})

In [ ]:
p_mid

In [ ]:
xout = pd.concat([xout1,xout2])
xout = xout.rename(columns={'ForecastId':'ForecastId_Quantile'})
xout.pop('index')
xout

In [ ]:
sum(n<0 for n in xout.TargetValue)

In [ ]:
xout.TargetValue = [0 if i < 0 else i for i in xout.TargetValue]
pred2 = np.around(xout, decimals=0)

In [ ]:
sum(n<0 for n in pred2.TargetValue)


In [ ]:
pred2

In [ ]:
pred2.to_csv('submission.csv', index=False)
print("Submission file Created.....")

In [ ]:
pred2 = pred2.sort_values(by='ForecastId_Quantile', ascending=False)
pred2